In [1]:
from __future__ import print_function
 
from keras.preprocessing import sequence
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation,Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D,GRU,Conv1D, MaxPooling1D,SpatialDropout1D
from keras.layers.recurrent import LSTM,GRU
from keras.layers.wrappers import Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from keras.layers import CuDNNGRU
from keras.layers.merge import concatenate, add
from keras.layers import Input,Reshape,Conv2D,MaxPool2D, concatenate

from keras.utils import multi_gpu_model
from keras.models import model_from_json
import numpy as np
import json
import requests
import pandas as pd

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
df_all = pd.read_csv('data/19999_test_question.csv')
print(f"df_all shape : {df_all.shape}")

df_all shape : (27, 4)


In [4]:
mapping = df_all.category.astype('category').cat.categories
mapping

Index(['Outlook與郵件', 'SAP', 'SAP ', '其他', '網路', '資訊系統'], dtype='object')

In [5]:
from NLP_JiebaSegmentor import JiebaSegmentor
jieba_dict_path1 = "jieba/dict_taiwan.txt"
jieba_dict_path2 = "jieba/userdict.txt"
jieba_dict_path3 = "jieba/dict.txt.big"
jieba_dict_path4 = "jieba/dict.txt.small"
jieba_dict_path5 = "jieba/dict_system.txt"
jieba_stopwords_path = "jieba/stopwords.txt"

js = JiebaSegmentor(dict_path=jieba_dict_path1,
                    userdict=[jieba_dict_path2,
                              jieba_dict_path3,
                              jieba_dict_path4
                              #jieba_dict_path5
                             ],
                    stopwords=True,
                    stopwords_path=jieba_stopwords_path)

Building prefix dict from /project/at081-group3/Project_19999/19999_2RNN/jieba/dict_taiwan.txt ...
Loading model from cache /tmp/jieba.u5f4d4d2a4ebf6cacbf248ccc4ee696aa.cache
Loading model cost 0.510 seconds.
Prefix dict has been built succesfully.


In [6]:
test_cut_raw_0 = js.lcut('FTP無法安裝', return_type='pandas')
test_cut_raw_0

,word
0,FTP
1,無法
2,安裝


In [7]:
# x_train=np.loadtxt("x_train.txt",dtype=int)
# y_train=np.loadtxt("y_train.txt",dtype=int)
 
# indices = np.arange(x_train.shape[0])
# np.random.shuffle(indices)
# x_train = x_train[indices]
# y_train = y_train[indices]
 
# print('Loading data...')
# #x_train=np.loadtxt("x_train.txt",dtype=int)
# #y_train=np.loadtxt("y_train.txt",dtype=int)
# x_test=x_train[20000:]
# y_test=y_train[20000:]
# x_train=x_train[:20000]
# y_train=y_train[:20000]
# #x_test=x_train
# #y_test=y_train
# print(len(x_train), 'train sequences')
# print(len(x_test), 'test sequences')
# print(x_train[:1])
# print('Pad sequences (samples x time)')

In [8]:
df_all.head()

,description,category,category_target,description_clean
0,"218無法登入,logon失敗",SAP,1,218logon失敗
1,無線網路不通,網路,3,無線網路不通
2,我的outlook薪資單打不開 # 19810,Outlook與郵件,0,我的outlook薪資單
3,MRO操作問題email:charles.huang@quantatw.com,資訊系統,4,mro
4,有關非法軟體安裝問題想詢問,其他,2,非法軟體安裝問題


In [9]:
# w_pandas = js.lcut('無法會員登入 顯示訊息 無效的帳號或不存在 帳號'.strip(), return_type='pandas')
# w_list = list(w_pandas['word'])
# w_list

In [10]:
def cut_to_word(s):
    blCondition=True
    w_pandas = js.lcut(s, return_type='pandas')
    w_list = list(w_pandas['word'])
    combie = ''
    combieNO=''
    beforeWord=''
    blAdd=True
    for i,w in enumerate(w_list):
        if w is ' ':
            continue
        conditionWord = str(combieNO) + str(w)
        
        if combieNO != '' and len(w)>1:# and len(conditionWord) == 4:
            combie = combie + w +','
            combie = combie+combieNO+w
            #print(combieNO+w)
        else:        
            #87.2%=>
            if w in ['修改','顯示','作業','申請單','問題','系統'] and beforeWord!='' and blCondition and len(w)>1:
                combie = combie +w+','
                combie = combie+beforeWord+w
            else:
                if w not in ['無法'] and len(w)>1:
                    combie = combie + w
                else:
                    blAdd=False
                 
        if i < len(w_list) - 1 and combie !='' and blAdd:
            combie = combie + ','
            
        blAdd = True
        if w in ['無法','不能'] and blCondition:#,'安裝','編輯','刪除','新增','作業','忘記','系統','發生','作業','下載','上傳','開啟','安裝']
            combieNO = w
        else:
            combieNO = ''
        beforeWord=w
    return combie    

# print (cut_to_word('可以幫我聯絡'.strip()))
df_all['cut_words'] = df_all['description_clean'].apply(lambda s: cut_to_word(s.strip()))

In [11]:
# 資料打散
df_all = df_all.sample(frac=1).reset_index(drop=True)
df_all.head()

,description,category,category_target,description_clean,cut_words
0,HP2014列印會出現亂碼,其他,2,hp列印會出現亂碼,"hp,列印,出現,亂碼"
1,電腦變很慢是不是中毒?,其他,2,電腦變很慢是中毒,"電腦,很慢,中毒"
2,異常SO單簽核有問題,資訊系統,4,異常so單簽核有問題,"異常,so,簽核,問題,簽核問題"
3,採購單簽核時發生錯誤!!!!,資訊系統,4,採購單簽核時發生錯誤,"採購單,簽核,發生,錯誤"
4,資訊需求單無法申請,資訊系統,4,資訊需求單,"資訊,需求單"


In [12]:
# 切分資料
X_train = df_all['cut_words']
y_train = df_all['category_target']

In [13]:
X_train.head()

0         hp,列印,出現,亂碼
1            電腦,很慢,中毒
2    異常,so,簽核,問題,簽核問題
3        採購單,簽核,發生,錯誤
4              資訊,需求單
Name: cut_words, dtype: object

In [14]:
# label 做 onehot
y_train_one_hot = np_utils.to_categorical(y_train)
print (y_train_one_hot[0])
# 總共類別數
num_classes = len((y_train_one_hot[0]))
print ('num_classes = {}'.format(num_classes))

[0. 0. 1. 0. 0.]
num_classes = 5


In [15]:
# # 標點符號過濾
# WORD_FILTERS = '!"#$&()*+,-./:;<=>?@[\\]^_{|}~\t\n'
# 字典數量
NUM_WORDS = 5001
# 向量長度
MAX_LEN = 30

token_intent = Tokenizer(num_words=NUM_WORDS,
                        split=",")
# token_intent = Tokenizer(filters=WORD_FILTERS,
#                         num_words=NUM_WORDS,
#                         split=",")

# 斷詞後全部合併丟到fit_on_texts,組出字典
token_intent.fit_on_texts(X_train)

def preprocessing(x, max_len):

    #將文字轉為數字序列
    train_seq_intent = token_intent.texts_to_sequences(x)
    
    #print(char_index)
    # 截長補短，讓所有數字序列長度一樣
    data = sequence.pad_sequences(train_seq_intent, maxlen=MAX_LEN)
    # print train_data_intent.shape
    return data

x_train = preprocessing(X_train, max_len=MAX_LEN)
char_index = token_intent.word_index

In [16]:
x_train

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 11, 12,  6,  3],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  4, 13,  7],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0, 14, 15,  1,  2, 16],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 17,  1, 18, 19],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 20, 21],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 22],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 23],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  

In [17]:
from keras import models 
model_path = '{}.h5'.format('19999_rnn')
model=models.load_model(model_path)

In [19]:
def to_cat_name(x): 
        return mapping[x]
    
def predict(test):
#     model = load_model(model_path)
    y_predict_probability = model.predict(test)
    y_predict = model.predict_classes(test)

    return to_cat_name(y_predict), y_predict, y_predict_probability

def predict_class(data):
    y_predict_probability = model.predict(data, batch_size=64, verbose=1)
    predict_arr = []
    predictClass = []
    for row in y_predict_probability: 
            classIndex=0
            selectClass=0
            selectProbability=0
            for item in row: 
                #print(float(item))
                if(selectProbability <= float(item)):
                    #print(classIndex)
                    selectProbability=float(item)
                    selectClass=classIndex
                classIndex=classIndex+1
            predictClass.append(selectClass)
            
    return to_cat_name(predictClass),predictClass,y_predict_probability

y_predict_name, y_predict, y_predict_probability = predict_class(x_train)

ResourceExhaustedError: OOM when allocating tensor with shape[30,27,32] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: bidirectional_6/transpose_2 = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](bidirectional_6/ReverseV2, bidirectional_6/transpose/perm)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: dense_15/Softmax/_357 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_325_dense_15/Softmax", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


In [ ]:
answer_name = to_cat_name(y_val)

In [ ]:
#model.predict_classes(x_val)

In [ ]:
X_train

In [165]:
mapping

Index(['Outlook與郵件', 'SAP', '其他', '網路', '資訊系統'], dtype='object')

In [166]:
# model.predict(x_val)

In [167]:
# predict_df = pd.DataFrame({'1_sentence':df_test.sentence,
#               '2_y_predict':y_predict,
#               '3_y_predict_name':y_predict_name,
#               '4_answer':df_test.target,
#               '5_y_predict_probability':list(y_predict_probability)})
# predict_df

In [168]:
predict_df = pd.DataFrame({
              '1_sentence':X_train,
              '2_y_predict':y_predict,
              '3_y_predict_name':y_predict_name,
              '4_answer':y_train,
              '5_answer_name':answer_name,
              '6_y_predict_probability':list(y_predict_probability)})
# predict_df

In [169]:
# accuracy
accuracy = 1 - (float(len(predict_df[predict_df['2_y_predict'] != predict_df['4_answer']])))/(len(predict_df))
print ('accuracy : {} !!!!!!!!'.format(accuracy))

accuracy : 0.844561403508772 !!!!!!!!


In [170]:
predict_df[predict_df['2_y_predict'] != predict_df['4_answer']]

,1_sentence,2_y_predict,3_y_predict_name,4_answer,5_answer_name,6_y_predict_probability
6601,"開啟,ie,出現,要求,輸入,access,code,訊息",2,其他,3,網路,"[0.08697885, 0.13981782, 0.3389513, 0.16366155..."
8471,"levellogin,fail",3,網路,1,SAP,"[0.028763168, 0.31563962, 0.06268179, 0.567144..."
9363,"之前,進入,問題,進,eps,data,folder,顯示,問題",0,Outlook與郵件,2,其他,"[0.67808056, 0.0069583566, 0.07864304, 0.21070..."
9369,"下週,到,外面,去,參加,研討會,一臺,筆電,需要,協助,安裝,win7office,環境,設定",3,網路,2,其他,"[0.00011890863, 0.00093693996, 0.0043625957, 0..."
3779,"之前,簽過,表單,這幾天,部門,代號,換了,扣帳,說,扣,不知,進行",4,資訊系統,1,SAP,"[5.9342792e-08, 0.0111106895, 4.8651968e-05, 1..."
7945,"報修,需要,新增,印表機",4,資訊系統,1,SAP,"[0.00067867397, 0.16627419, 0.01005499, 0.0046..."
2342,"登入後,正常,使用",3,網路,1,SAP,"[0.026793843, 0.06477459, 0.037382793, 0.84839..."
5547,"qct,離職,留下,一臺,nb,想收,outlook,信箱,網路,連線",0,Outlook與郵件,3,網路,"[0.9966943, 3.6157e-05, 0.0029007131, 0.000361..."
3193,"操作,錯誤",1,SAP,4,資訊系統,"[0.22711654, 0.5196952, 0.10772075, 0.06515804..."
5107,"郵件,收發,記得,新人,訓練,qex,學習,互動,平臺,出現,需要,登入,帳號密碼,小,視窗...",4,資訊系統,0,Outlook與郵件,"[3.4491986e-06, 0.0018475467, 0.0015475154, 0...."


In [83]:
# confusion matrix
pd.crosstab(np.array(y_predict_name), np.array(answer_name),
            rownames=['predict'], colnames=['answer'])

answer,Outlook與郵件,SAP,其他,網路,資訊系統
predict,,,,,
Outlook與郵件,773,8,17,20,16
SAP,5,289,10,13,8
其他,4,17,129,39,23
網路,23,26,63,1083,24
資訊系統,7,16,21,13,203


In [49]:

# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# model.save_weights("model.h5")
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")
# loaded_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# score = loaded_model.evaluate(x_test, y_test, verbose=0)
# print(score)